# Exercise05 : Distributed Training with Curated Environments

Here we change our sample (see "[Exercise03 : Just Train in Your Working Machine](https://github.com/tsmatz/azureml-tutorial-tensorflow-v1/blob/master/notebooks/exercise03_train_simple.ipynb)") for distributed training using multiple machines.

Azure Machine Learning provides built-in pre-configured image, called curated environments. (See [here](https://docs.microsoft.com/en-us/azure/machine-learning/resource-curated-environments) for the list of curated environments.)<br>
You can run distributed training by Horovod using curated environment ```AzureML-TensorFlow-1.13-CPU```, and here I configure using this curated environment.

In this exercise, we use Horovod framework using built-in pre-configured image, called curated environments. (See [here](https://docs.microsoft.com/en-us/azure/machine-learning/resource-curated-environments) for the list of curated environments.)<br>
As you saw in previous [Exercise04](https://github.com/tsmatz/azureml-tutorial-tensorflow-v1/blob/master/notebooks/exercise04_train_remote.ipynb), you can also configure distributed training manually using primitive ```azureml.core.ScriptRunConfig``` without curated environments. (See [here](https://tsmatz.wordpress.com/2019/01/17/azure-machine-learning-service-custom-amlcompute-and-runconfig-for-mxnet-distributed-training/) for the manually configured example using ```azureml.core.ScriptRunConfig```.)

*back to [index](https://github.com/tsmatz/azureml-tutorial-tensorflow-v1/)*

## Save your training script as file (train.py)

Create ```scirpt``` directory.

In [1]:
import os
script_folder = './script'
os.makedirs(script_folder, exist_ok=True)

Change our original source code ```train.py``` (see "[Exercise03 : Just Train in Your Working Machine](https://github.com/tsmatz/azureml-tutorial-tensorflow-v1/blob/master/notebooks/exercise03_train_simple.ipynb)") as follows. The lines commented "##### modified" is modified lines.    
After that, please add the following ```%%writefile``` at the beginning of the source code and run this cell.    
This source code is saved as ```./script/train_horovod.py```.

In [2]:
%%writefile script/train_horovod.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
import os
import shutil
import argparse
import math

import tensorflow as tf
import horovod.tensorflow as hvd ##### modified

FLAGS = None
batch_size = 100

#
# define functions for Estimator
#

def _my_input_fn(filepath, num_epochs):
    # image - 784 (=28 x 28) elements of grey-scaled integer value [0, 1]
    # label - digit (0, 1, ..., 9)
    data_queue = tf.train.string_input_producer(
        [filepath],
        num_epochs = num_epochs) # data is repeated and it raises OutOfRange when data is over
    data_reader = tf.TFRecordReader()
    _, serialized_exam = data_reader.read(data_queue)
    data_exam = tf.parse_single_example(
        serialized_exam,
        features={
            'image_raw': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64)
        })
    data_image = tf.decode_raw(data_exam['image_raw'], tf.uint8)
    data_image.set_shape([784])
    data_image = tf.cast(data_image, tf.float32) * (1. / 255)
    data_label = tf.cast(data_exam['label'], tf.int32)
    data_batch_image, data_batch_label = tf.train.batch(
        [data_image, data_label],
        batch_size=batch_size)
    return {'inputs': data_batch_image}, data_batch_label

def _get_input_fn(filepath, num_epochs):
    return lambda: _my_input_fn(filepath, num_epochs)

def _my_model_fn(features, labels, mode):
    # with tf.device(...): # You can set device if using GPUs

    # define network and inference
    # (simple 2 fully connected hidden layer : 784->128->64->10)
    with tf.name_scope('hidden1'):
        weights = tf.Variable(
            tf.truncated_normal(
                [784, FLAGS.first_layer],
                stddev=1.0 / math.sqrt(float(784))),
            name='weights')
        biases = tf.Variable(
            tf.zeros([FLAGS.first_layer]),
            name='biases')
        hidden1 = tf.nn.relu(tf.matmul(features['inputs'], weights) + biases)
    with tf.name_scope('hidden2'):
        weights = tf.Variable(
            tf.truncated_normal(
                [FLAGS.first_layer, FLAGS.second_layer],
                stddev=1.0 / math.sqrt(float(FLAGS.first_layer))),
            name='weights')
        biases = tf.Variable(
            tf.zeros([FLAGS.second_layer]),
            name='biases')
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal(
                [FLAGS.second_layer, 10],
                stddev=1.0 / math.sqrt(float(FLAGS.second_layer))),
        name='weights')
        biases = tf.Variable(
            tf.zeros([10]),
            name='biases')
        logits = tf.matmul(hidden2, weights) + biases
 
    # compute evaluation matrix
    predicted_indices = tf.argmax(input=logits, axis=1)
    if mode != tf.estimator.ModeKeys.PREDICT:
        label_indices = tf.cast(labels, tf.int32)
        accuracy = tf.metrics.accuracy(label_indices, predicted_indices)
        tf.summary.scalar('accuracy', accuracy[1]) # output to TensorBoard
 
        loss = tf.losses.sparse_softmax_cross_entropy(
            labels=labels,
            logits=logits)
 
    # define operations
    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step = tf.train.get_or_create_global_step()        
        optimizer = tf.train.GradientDescentOptimizer(
            learning_rate=FLAGS.learning_rate)
        optimizer = hvd.DistributedOptimizer(optimizer) ##### modified
        train_op = optimizer.minimize(
            loss=loss,
            global_step=global_step)
        return tf.estimator.EstimatorSpec(
            mode,
            loss=loss,
            train_op=train_op)
    if mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
            'accuracy': accuracy
        }
        return tf.estimator.EstimatorSpec(
            mode,
            loss=loss,
            eval_metric_ops=eval_metric_ops)
    if mode == tf.estimator.ModeKeys.PREDICT:
        probabilities = tf.nn.softmax(logits, name='softmax_tensor')
        predictions = {
            'classes': predicted_indices,
            'probabilities': probabilities
        }
        export_outputs = {
            'prediction': tf.estimator.export.PredictOutput(predictions)
        }
        return tf.estimator.EstimatorSpec(
            mode,
            predictions=predictions,
            export_outputs=export_outputs)

def _my_serving_input_fn():
    inputs = {'inputs': tf.placeholder(tf.float32, [None, 784])}
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)

#
# Main
#

parser = argparse.ArgumentParser()
parser.add_argument(
    '--data_folder',
    type=str,
    default='./data',
    help='Folder path for input data')
parser.add_argument(
    '--chkpoint_folder',
    type=str,
    default='./logs',  # AML experiments logs folder
    help='Folder path for checkpoint files')
parser.add_argument(
    '--model_folder',
    type=str,
    default='./outputs',  # AML experiments outputs folder
    help='Folder path for model output')
parser.add_argument(
    '--learning_rate',
    type=float,
    default='0.07',
    help='Learning Rate')
parser.add_argument(
    '--first_layer',
    type=int,
    default='128',
    help='Neuron number for the first hidden layer')
parser.add_argument(
    '--second_layer',
    type=int,
    default='64',
    help='Neuron number for the second hidden layer')
FLAGS, unparsed = parser.parse_known_args()

# clean checkpoint and model folder if exists
if os.path.exists(FLAGS.chkpoint_folder) :
    for file_name in os.listdir(FLAGS.chkpoint_folder):
        file_path = os.path.join(FLAGS.chkpoint_folder, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
if os.path.exists(FLAGS.model_folder) :
    for file_name in os.listdir(FLAGS.model_folder):
        file_path = os.path.join(FLAGS.model_folder, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)

hvd.init() ##### modified

# read TF_CONFIG
run_config = tf.estimator.RunConfig()

# create Estimator
mnist_fullyconnected_classifier = tf.estimator.Estimator(
    model_fn=_my_model_fn,
    model_dir=FLAGS.chkpoint_folder if hvd.rank() == 0 else None, ##### modified
    config=run_config)
train_spec = tf.estimator.TrainSpec(
    input_fn=_get_input_fn(os.path.join(FLAGS.data_folder, 'train.tfrecords'), 2),
    #max_steps=60000 * 2 / batch_size)
    max_steps=(60000 * 2 / batch_size) // hvd.size(), ##### modified
    hooks=[hvd.BroadcastGlobalVariablesHook(0)]) ##### modified
eval_spec = tf.estimator.EvalSpec(
    input_fn=_get_input_fn(os.path.join(FLAGS.data_folder, 'test.tfrecords'), 1),
    steps=10000 * 1 / batch_size,
    start_delay_secs=0)

# run !
tf.estimator.train_and_evaluate(
    mnist_fullyconnected_classifier,
    train_spec,
    eval_spec
)

# save model and variables
if hvd.rank() == 0 : ##### modified
    model_dir = mnist_fullyconnected_classifier.export_savedmodel(
        export_dir_base = FLAGS.model_folder,
        serving_input_receiver_fn = _my_serving_input_fn)
    print('current working directory is ', os.getcwd())
    print('model is saved ', model_dir)

Writing script/train_horovod.py


## Train on multiple machines (Horovod)

### Step 1 : Get workspace setting

Before starting, you must read your configuration settings. (See "[Exercise01 : Prepare Config Settings](https://github.com/tsmatz/azureml-tutorial-tensorflow-v1/blob/master/notebooks/exercise01_prepare_config.ipynb)".)

In [3]:
from azureml.core import Workspace
import azureml.core

ws = Workspace.from_config()

### Step 2 : Create multiple virtual machines (cluster)

Create your new AML compute for distributed clusters. By enabling auto-scaling from 0 to 4, you can save money (all nodes are terminated) if it's inactive.    
If already exists, this script will get the existing cluster.

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=ws, name='mycluster01')
    print('found existing:', compute_target.name)
except ComputeTargetException:
    print('creating new.')
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_D2_v2',
        min_nodes=0,
        max_nodes=3)
    compute_target = ComputeTarget.create(ws, 'mycluster01', compute_config)
    compute_target.wait_for_completion(show_output=True)

creating new.
InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
# get a status for the current cluster.
print(compute_target.status.serialize())

{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-07-12T02:11:06.030000+00:00', 'errors': None, 'creationTime': '2021-07-12T02:10:43.391839+00:00', 'modifiedTime': '2021-07-12T02:11:09.118976+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 3, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


### Step 3 : Get dataset reference for files

You can mount your registered dataset (See "[Exercise02 : Prepare Data](https://github.com/tsmatz/azureml-tutorial-tensorflow-v1/blob/master/notebooks/exercise02_prepare_data.ipynb)") into your AML compute.<br>
Now we get the registered dataset reference.

In [6]:
from azureml.core import Dataset

dataset = Dataset.get_by_name(ws, 'mnist_tfrecords_dataset', version='latest')

# # For using unregistered data, see below
# from azureml.core import Datastore
# from azureml.core import Dataset
# ds = ws.get_default_datastore()
# ds_paths = [(ds, 'tfdata/')]
# dataset = Dataset.File.from_files(path = ds_paths)

### Step 4 : Configure with Curated Environment

Unlike the previous exercise (see "[Exercise04 : Train on Remote GPU Virtual Machine](https://github.com/tsmatz/azureml-tutorial-tensorflow-v1/blob/master/notebooks/exercise04_train_remote.ipynb)"), here I use the built-in curated environment.    
Azure Machine Learning provides pre-configured (built-in) image, called **curated environments**, for a variety of purposes. (See [here](https://docs.microsoft.com/en-us/azure/machine-learning/resource-curated-environments) for the list of curated environments.)<br>
Here we run distributed training by Horovod and TensorFlow 1.x using a curated environment ```AzureML-TensorFlow-1.13-CPU```.

In [7]:
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import MpiConfiguration

tf_env = Environment.get(workspace=ws, name='AzureML-TensorFlow-1.13-CPU')
src = ScriptRunConfig(
    source_directory='./script',
    script='train_horovod.py',
    arguments=['--data_folder', dataset.as_mount()],
    compute_target=compute_target,
    environment=tf_env,
    distributed_job_config=MpiConfiguration(node_count=3))

[Optional] This ```AzureML-TensorFlow-1.13-CPU``` includes Horovod 0.16.1.    
When you want to see the packages included in this curated environment, please run as follows and see the saved configuration.

In [8]:
tf_env.save_to_directory(path='AzureML-TensorFlow-1.13-CPU')

### Step 5 : Run script and wait for completion

In [9]:
from azureml.core import Experiment

exp = Experiment(workspace=ws, name='tf_distribued')
run = exp.submit(config=src)
run.wait_for_completion(show_output=True)

RunId: tf_distribued_1626057509_1c54bf47
Web View: https://ml.azure.com/runs/tf_distribued_1626057509_1c54bf47?wsid=/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourcegroups/TEST20210712/workspaces/ws01&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/55_azureml-execution-tvmps_50ef9d914602729f6f3634241e9517a271ba8f7237cb370b0e6ac6359bf55006_d.txt

2021-07-12T02:38:46Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/ws01/azureml/tf_distribued_1626057509_1c54bf47/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/ws01/azureml/tf_distribued_1626057509_1c54bf47/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=88426 -o nonempty -o allow_other --config-file=/mnt/batch/tasks/shared/LS_root/jobs/ws01/azureml/tf_distribued_1626057509_1c54bf47/configs/workspaceblobstore.cfg --log-level=LOG_WARNING
2021-07-12T02:38:46Z Successfully mounted a/an Blobfuse File System at

Instructions for updating:
Use standard file APIs to check for files with this prefix.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
current working directory is  /mnt/batch/tasks/shared/LS_root/jobs/ws01/azureml/tf_distribued_1626057509_1c54bf47/wd/azureml/tf_distribued_1626057509_1c54bf47
model is saved  b'./outputs/1626057567'


[2021-07-12T02:39:27.789152] The experiment completed successfully. Finalizing run...
Cleaning up all outstanding Run operations, waiting 900.0 seconds
1 items cleaning up...
Cleanup took 0.06380271911621094 seconds
[2021-07-12T02:39:27.993863] Finished context manager injector.

Streaming azureml-logs/75_job_post-tvmps_50ef9d914602729f6f3634241e9517a271ba8f7237cb370b0e6ac6359bf55006_d.txt

[2021-07-12T02:39:30.428088] Entering job release
[2021-07-12T02:39:31.380676] Starting job release
[2021-07-12T02:39:

{'runId': 'tf_distribued_1626057509_1c54bf47',
 'target': 'mycluster01',
 'status': 'Completed',
 'startTimeUtc': '2021-07-12T02:38:43.83731Z',
 'endTimeUtc': '2021-07-12T02:39:44.94448Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'd4d953d9-a970-4bfe-93cd-4aecf9df51e7',
  'azureml.git.repository_uri': 'https://github.com/tsmatz/azureml-tutorial-tensorflow-v1.git',
  'mlflow.source.git.repoURL': 'https://github.com/tsmatz/azureml-tutorial-tensorflow-v1.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': 'b5e83f1c910ce7bde6489279614a6dace708dfed',
  'mlflow.source.git.commit': 'b5e83f1c910ce7bde6489279614a6dace708dfed',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json',
  'azureml.RuntimeType': ''},
 'inputDatasets': [{'dataset': {'id': 'f3adcd26-eecb-4121-8cb8-b4ca49728bee'}, 'consumptionDetails': {'typ

### Step 6 : Check results

In [10]:
run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_50ef9d914602729f6f3634241e9517a271ba8f7237cb370b0e6ac6359bf55006_d.txt',
 'azureml-logs/55_azureml-execution-tvmps_5fe2606f0ee22c929efd7b00005abd84f5ab562b31a700602884c04423a4eadd_d.txt',
 'azureml-logs/55_azureml-execution-tvmps_6e40f4ce95034c8811263bcf3983d6bddec7e5caddd8d2a2fd424c961c178f84_d.txt',
 'azureml-logs/65_job_prep-tvmps_50ef9d914602729f6f3634241e9517a271ba8f7237cb370b0e6ac6359bf55006_d.txt',
 'azureml-logs/65_job_prep-tvmps_5fe2606f0ee22c929efd7b00005abd84f5ab562b31a700602884c04423a4eadd_d.txt',
 'azureml-logs/65_job_prep-tvmps_6e40f4ce95034c8811263bcf3983d6bddec7e5caddd8d2a2fd424c961c178f84_d.txt',
 'azureml-logs/70_driver_log_0.txt',
 'azureml-logs/70_driver_log_1.txt',
 'azureml-logs/70_driver_log_2.txt',
 'azureml-logs/70_mpi_log.txt',
 'azureml-logs/75_job_post-tvmps_50ef9d914602729f6f3634241e9517a271ba8f7237cb370b0e6ac6359bf55006_d.txt',
 'azureml-logs/75_job_post-tvmps_5fe2606f0ee22c929efd7b00005abd84f5ab562b31a700602884c04

**Please change ```1626057567``` to meet previous results.**

In [11]:
run.download_file(
    name='outputs/1626057567/saved_model.pb',
    output_file_path='distributed_model/saved_model.pb')
run.download_file(
    name='outputs/1626057567/variables/variables.data-00000-of-00001',
    output_file_path='distributed_model/variables/variables.data-00000-of-00001')
run.download_file(
    name='outputs/1626057567/variables/variables.index',
    output_file_path='distributed_model/variables/variables.index')

In [12]:
import tensorflow as tf

# Read data by tensor
tfdata = tf.data.TFRecordDataset('./data/test.tfrecords')
iterator = tf.compat.v1.data.make_one_shot_iterator(tfdata)
data_org = iterator.get_next()
data_exam = tf.parse_single_example(
    data_org,
    features={
        'image_raw': tf.FixedLenFeature([], tf.string),
        'label': tf.FixedLenFeature([], tf.int64)
    })
data_image = tf.decode_raw(data_exam['image_raw'], tf.uint8)
data_image.set_shape([784])
data_image = tf.cast(data_image, tf.float32) * (1. / 255)
data_label = tf.cast(data_exam['label'], tf.int32)

# Run tensor and generate data
with tf.Session() as sess:
    image_arr = []
    label_arr = []
    for i in range(3):
        image, label = sess.run([data_image, data_label])
        image_arr.append(image)
        label_arr.append(label)

# Predict
pred_fn = tf.contrib.predictor.from_saved_model('./distributed_model')
pred = pred_fn({'inputs': image_arr})

print('Predicted: ', pred['classes'].tolist())
print('Actual   : ', label_arr)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ./distributed_model/variables/variables
Predicted:  [7, 2, 1]
Actual   :  [7, 2, 1]


### Step 7 : Remove AML compute

**You don't need to remove your AML compute** for saving money, because the nodes will be automatically terminated, when it's inactive.    
But if you want to clean up, please run the following.

In [13]:
# Delete cluster (nbodes) and remove from AML workspace
mycompute = AmlCompute(workspace=ws, name='mycluster01')
mycompute.delete()